<a href="https://colab.research.google.com/github/JalpaK88/Projects_SQL/blob/main/SQL_bigquery_global_air_quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.cloud import bigquery

# Step 1: Authenticate
auth.authenticate_user()

# Step 2: Prompt user for project ID
PROJECT_ID = input("Enter your GCP project ID: ").strip()

# Check if a project ID was entered
if not PROJECT_ID:
    raise ValueError("You must enter a valid GCP project ID!")

# Step 3: Initialize BigQuery client with the provided project ID
client = bigquery.Client(project=PROJECT_ID)
print("Using project:", client.project)

dataset_ref_2 = client.dataset("openaq", project="bigquery-public-data")
dataset_2 = client.get_dataset(dataset_ref_2)
table_ref = dataset_ref_2.table("global_air_quality")
table = client.get_table(table_ref)
rows= client.list_rows(table,max_results=5).to_dataframe()
rows





Enter your GCP project ID: brilliant-flame-471022-p3
Using project: brilliant-flame-471022-p3


,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


In [ ]:
# # Query to select all the items from the "city" column where the "country" column is 'US'
# query = """
# select city
# from bigquery-public-data.openaq.global_air_quality
# where country = 'US'
# """
# Query to select all the items from the "city" column where the "country" column is 'US'
query = """
select city, country
from bigquery-public-data.openaq.global_air_quality
where country = 'US'
"""

# # Query to select all the items from the "city" column where the "country" column is 'US'
# query = """
# select *
# from bigquery-public-data.openaq.global_air_quality
# where country = 'US'
# """

In [ ]:
# client = bigquery.Client()

In [ ]:
query_job = client.query(query)

In [ ]:
# us_cities = query_job.to_dataframe()
us_cities_countries = query_job.to_dataframe()

In [ ]:
# us_cities.city.value_counts().head()
us_cities_countries.city.value_counts().head()
# us_cities_countries.country.value_counts().head()

,count
city,
Phoenix-Mesa-Scottsdale,39414
Los Angeles-Long Beach-Santa Ana,27479
Riverside-San Bernardino-Ontario,26887
New York-Northern New Jersey-Long Island,25417
San Francisco-Oakland-Fremont,22710


In [ ]:
# Top 5 most frequent cities
print("Top 5 US cities:")
print(us_cities_countries["city"].value_counts().head())

# (Optional) Top 5 most frequent countries in the result
print("\nTop 5 countries (should all be US here):")
print(us_cities_countries["country"].value_counts().head())

Top 5 US cities:
city
Phoenix-Mesa-Scottsdale                     39414
Los Angeles-Long Beach-Santa Ana            27479
Riverside-San Bernardino-Ontario            26887
New York-Northern New Jersey-Long Island    25417
San Francisco-Oakland-Fremont               22710
Name: count, dtype: int64

Top 5 countries (should all be US here):
country
US    1421351
Name: count, dtype: int64


In [ ]:
# Step 4: SQL query to get countries reporting pollution in 'ppm'
first_query = """
SELECT DISTINCT country
FROM `bigquery-public-data.openaq.global_air_quality`
WHERE unit = 'ppm'
LIMIT 10
"""

# Step 5: Safe query config (limit bytes billed)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

# Step 6: Run query
query_job = client.query(first_query, job_config=safe_config)

# Step 7: Convert results to Pandas DataFrame
first_results = query_job.to_dataframe()

# Step 8: Show results
print(first_results)

  country
0      AR
1      TW
2      IL
3      CO
4      EC
5      RW
6      AU
7      BR
8      CA
9      MX


In [ ]:
# Which pollution levels were reported to be exactly 0?

# Set zero_pollution_query to select all columns of the rows where the value column is 0.
# Set zero_pollution_results to a pandas DataFrame containing the query results.

In [ ]:
zero_pollution_query = """
                       SELECT *
                       FROM `bigquery-public-data.openaq.global_air_quality`
                       WHERE value = 0
                       """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(zero_pollution_query, job_config=safe_config)

zero_pollution_results = query_job.to_dataframe()
print(zero_pollution_results.head())

                       location          city country pollutant  value  \
0    Żary, ul. Szymanowskiego 8          Żary      PL        bc    0.0   
1       Starachowice, ul. Złota  Starachowice      PL        bc    0.0   
2    Żary, ul. Szymanowskiego 8          Żary      PL        bc    0.0   
3  Koszalin, ul. Armii Krajowej      Koszalin      PL        bc    0.0   
4       Starachowice, ul. Złota  Starachowice      PL        bc    0.0   

                  timestamp   unit source_name  latitude  longitude  \
0 2022-05-18 15:00:00+00:00  µg/m³        GIOS       1.0  51.642656   
1 2022-05-07 11:00:00+00:00  µg/m³        GIOS       1.0  51.050611   
2 2022-05-04 16:00:00+00:00  µg/m³        GIOS       1.0  51.642656   
3 2022-05-17 14:00:00+00:00  µg/m³        GIOS       1.0  54.193986   
4 2022-05-15 14:00:00+00:00  µg/m³        GIOS       1.0  51.050611   

   averaged_over_in_hours       location_geom  
0               15.127808  POINT(51.642656 1)  
1               21.084175  POINT